### Load the model

In [1]:
import re
import random
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use cpu


### Create the seat map

In [2]:
def generate_seat_labels(rows=6, cols=8):
    return {f"{chr(65 + r)}{c+1}" for r in range(rows) for c in range(cols)}



This cell defines the core data used in the theater booking app, including available performances, seating availability, and sample bookings for testing.

In [3]:
theater_data = {
    "performances": [
        {"hall": "Hall 1", "play": "Antigone", "showtimes": ["18:00", "21:00"]},
        {"hall": "Hall 2", "play": "Electra", "showtimes": ["18:00", "21:00"]}
    ],
    "seat_availability": {
        ("Hall 1", "18:00"): generate_seat_labels(),
        ("Hall 1", "21:00"): generate_seat_labels(),
        ("Hall 2", "18:00"): generate_seat_labels(),
        ("Hall 2", "21:00"): generate_seat_labels(),
    },
   "bookings": {  #test sample
    "1": {
      "hall": "Hall 1",
      "showtime": "18:00",
      "num_tickets": 2,
      "seats": ["A1", "A2"],
      "name": "Eleni Markou",
      "phone": "11",
      "email": "eleni.markou@example.com",
      "paid": False
      }
   }
}

session_state = {
    "active": False,
    "current_step": None,
    "collected_data": {},
    "waiting_for_booking_id": None,
    "waiting_for_phone_number": None,
    "step_history": []  # Added to track previous steps
}
booking_steps = ["plays", "hall", "showtime", "tickets", "seats", "confirm_order", "payment", "confirm_payment", "collect_email"]

### Booking Function
Books tickets for a specific hall and showtime if enough seats are available. Stores booking info including customer details and payment status.


In [4]:
########################### BOOKING ##########################

def book_tickets(hall, showtime, num_tickets, seats, paid=False, name="Unknown", phone="Unknown", email="Unknown"):

    key = (hall, showtime)
    available_seats = theater_data["seat_availability"].get(key, set())

    if len(available_seats) >= num_tickets:
        theater_data["seat_availability"][key] -= set(seats)
        booking_id = len(theater_data["bookings"]) + 1
        theater_data["bookings"][booking_id] = {
            "hall": hall,
            "showtime": showtime,
            "num_tickets": num_tickets,
            "seats": seats,
            "name": name,
            "phone": phone,
            "email": email,
            "paid": paid
        }
        personal_info = f" for {name} (phone: {phone}, email: {email})" if name != "Unknown" else ""
        return f"Booked {num_tickets} tickets for {name} in {hall} at {showtime}. Seats: {', '.join(map(str, seats))}. Booking ID: {booking_id}."

    return "Not enough seats available."

### LLM Response Classification
Uses a text classification pipeline to label user input based on predefined categories. Returns the top label.


In [5]:
def llm_response(input_text,candidate_labels):
    result = pipe(input_text, candidate_labels)
    print(result)
    top_label = result['labels'][0].lower()
    return top_label


### Email Utility
Validates email format and sends plain text emails using Gmail's SMTP server.


In [6]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


def is_valid_email(email):
    # Basic regex for email validation
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None


def send_email(to_email, subject, body, from_email, from_password):
    msg = MIMEMultipart()
    msg["From"] = from_email
    msg["To"] = to_email
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "plain"))

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(from_email, from_password)
        server.send_message(msg)
        server.quit()
        return True
    except Exception as e:
        print(f"Error sending email: {e}")
        return False


### Booking Process Handler  
Manages user inputs step-by-step for booking tickets, including navigation, selection, confirmation, payment, and email delivery.


In [7]:

def handle_booking_process(user_input):
    global session_state

    exit_keywords = [ "stop", "exit", "quit"]
    if any(keyword in user_input.lower() for keyword in exit_keywords):
        session_state["active"] = False
        session_state["current_step"] = None
        session_state["collected_data"] = {}
        session_state["step_history"] = []
        return "Booking process cancelled. How can I assist you further?"

    def go_back():
        if session_state["step_history"]:
            session_state["current_step"] = session_state["step_history"].pop()
            return True
        else:
            session_state["active"] = False
            session_state["current_step"] = None
            return False

    # Check for "back" or "previous"
    if "back" in user_input.lower() or "previous" in user_input.lower():
        if go_back():
            return f"Going back to {session_state['current_step']} selection."
        else:
            return "No previous step. How can I assist you?"

    current_step = session_state["current_step"]

    if current_step == "plays":
        session_state["current_step"] = "hall"
        return list_available_plays(theater_data)

    elif current_step == "hall":
        # Access theater data
        performances = theater_data["performances"]
        play_options = {p["play"].lower(): p for p in performances}
        hall_options = {p["hall"].lower(): p for p in performances}

        options = (

            f", ".join([f"play: {p}" for p in play_options.keys()]) + ", " +
            ", ".join([f"hall: {h}" for h in hall_options.keys()]) + ", unknown"
        )

        # Get LLM response
        response = llm_response(user_input, options)
        print(response)

        # Parse LLM response
        if "play:" in response:
            play_name = response.split("play:")[1].strip()
            if play_name in play_options:
                hall = play_options[play_name]["hall"]
                session_state["collected_data"]["hall"] = hall
                session_state["collected_data"]["play"] = play_options[play_name]["play"]
                session_state["step_history"].append("hall")
                session_state["current_step"] = "showtime"
                return f"'{play_options[play_name]['play']}' in {hall} -> Showtimes: {', '.join(play_options[play_name]['showtimes'])}\nWhat time?"
        elif "hall:" in response:
            hall_name = response.split("hall:")[1].strip()
            if hall_name in hall_options:
                session_state["collected_data"]["hall"] = hall_options[hall_name]["hall"]
                session_state["collected_data"]["play"] = hall_options[hall_name]["play"]
                session_state["step_history"].append("hall")
                session_state["current_step"] = "showtime"
                return f"Showtimes in {hall_options[hall_name]['play']}: {', '.join(hall_options[hall_name]['showtimes'])}\nWhat time?"

        # Fallback if LLM fails to identify a valid play or hall
        all_plays = "\n".join(f"- {p['play']} in {p['hall']}" for p in performances)
        return f"Couldn't identify play or hall. Please specify a play or hall from the options:\n{all_plays}"

    elif current_step == "showtime":
        # Handle showtime selection
        time_match = re.search(r'(\d{1,2}:\d{2})', user_input)
        if time_match:
            time = time_match.group(1)
            hall = session_state["collected_data"]["hall"]
            available_times = [st for p in theater_data["performances"] if p["hall"] == hall for st in p["showtimes"]]
            if time in available_times:
                session_state["collected_data"]["time"] = time
                session_state["step_history"].append("showtime")
                session_state["current_step"] = "tickets"
                return "Great! How many tickets (1-100)?"
            return f"Invalid time. Available: {', '.join(available_times)}"
        return "Enter time in HH:MM format (e.g., 18:00):"

    elif current_step == "tickets":
        # Handle ticket quantity
        ticket_match = re.search(r'\d+', user_input)
        if ticket_match:
            num_tickets = int(ticket_match.group())
            if 1 <= num_tickets <= 100:
                session_state["collected_data"]["num_tickets"] = num_tickets
                session_state["step_history"].append("tickets")
                session_state["current_step"] = "seats"
                hall = session_state["collected_data"]["hall"]
                time = session_state["collected_data"]["time"]
                all_seats = generate_seat_labels()
                seats = sorted(all_seats )


                return f"Available seats:\n{', '.join(seats)}\nSelect {num_tickets} seats (e.g., 'A1,B2,C3' or 'A1-A3'):"
            return "Invalid quantity. Please enter a number between 1 and 100."
        return "Enter the number of tickets you want:"

    elif current_step == "seats":
        # Handle seat selection
        selected_seats = set()
        for part in user_input.split(','):
            part = part.strip().upper()
            if '-' in part:
                try:
                    start_label, end_label = part.split('-')
                    start_row, start_num = start_label[0], int(start_label[1:])
                    end_row, end_num = end_label[0], int(end_label[1:])
                    if start_row == end_row:
                        selected_seats.update({f"{start_row}{n}" for n in range(start_num, end_num + 1)})
                except ValueError:
                    pass
            elif re.match(r'^[A-Z]\d+$', part):
                selected_seats.add(part)

        hall = session_state["collected_data"]["hall"]
        time = session_state["collected_data"]["time"]
        available_seats = theater_data["seat_availability"].get((hall, time), set())

        if not selected_seats:
            return "Enter valid seat labels (e.g., 'A1,B2,C3' or 'A1-A3')."
        elif not selected_seats.issubset(available_seats):
            invalid = selected_seats - available_seats
            return f"Seats {', '.join(sorted(invalid))} are unavailable. Choose from: {', '.join(sorted(available_seats))}"
        elif len(selected_seats) != session_state["collected_data"]["num_tickets"]:
            return f"Selected {len(selected_seats)} seats, but you need {session_state['collected_data']['num_tickets']}. Please select exactly {session_state['collected_data']['num_tickets']} seats."

        session_state["collected_data"]["seats"] = sorted(selected_seats)
        session_state["step_history"].append("seats")
        session_state["current_step"] = "confirm_order"
        session_state["sub_step"] = "awaiting_confirmation"

        play = session_state["collected_data"]["play"]
        hall = session_state["collected_data"]["hall"]
        time = session_state["collected_data"]["time"]
        num_tickets = session_state["collected_data"]["num_tickets"]
        seats = ', '.join(session_state["collected_data"]["seats"])
        summary = f"Play: {play}\nHall: {hall}\nShowtime: {time}\nNumber of tickets: {num_tickets}\nSeats: {seats}"
        return f"Please confirm your order:\n{summary}\nJust double-checking, is everything in order with the booking data?"

    elif current_step == "confirm_order":
          sub_step = session_state.get("sub_step", "awaiting_confirmation")

          if sub_step == "awaiting_confirmation":

              if user_input in ["yes", "Yes", "YES", "perfect"]:
                  session_state["current_step"] = "payment"
                  del session_state["sub_step"]
                  return "Great! Let's proceed to payment. Choose payment method:\n- Credit Card\n- Cash"
              elif user_input in ["no","No", "NO","nop"]:
                  session_state["sub_step"] = "awaiting_change"
                  return "What would you like to change? You can say 'play', 'hall', 'showtime', 'tickets', or 'seats'."
              else:
                  return "I'm sorry, I didn't understand that. Is everything correct with your order?"

          elif sub_step == "awaiting_change":

              candidate_labels = ["play", "hall", "showtime", "tickets", "seats", "unclear"]
              change_classification = llm_response(user_input, candidate_labels)

              if change_classification in ["play", "hall"]:
                  # Reset subsequent steps
                  for key in ["time", "num_tickets", "seats"]:
                      session_state["collected_data"].pop(key, None)
                  session_state["current_step"] = "hall"
                  del session_state["sub_step"]
                  return "Let's select a new play or hall."
              elif change_classification == "showtime":
                  for key in ["num_tickets", "seats"]:
                      session_state["collected_data"].pop(key, None)
                  session_state["current_step"] = "showtime"
                  del session_state["sub_step"]
                  return f"Let's select a new showtime."

              elif change_classification == "tickets":
                  session_state["collected_data"].pop("seats", None)
                  session_state["current_step"] = "tickets"
                  del session_state["sub_step"]
                  return "Let's select the number of tickets again."
              elif change_classification == "seats":
                  session_state["current_step"] = "seats"
                  del session_state["sub_step"]
                  return f"Let's select new seats."
              else:
                  session_state["sub_step"] = "awaiting_start_over"
                  return "I'm sorry, I didn't understand that. Would you like to start the booking from the beginning?"

          elif sub_step == "awaiting_start_over":

              candidate_labels = ["yes", "no"]
              start_over_classification = llm_response(user_input, candidate_labels)

              if start_over_classification == "yes":
                  session_state["active"] = False
                  session_state["current_step"] = None
                  session_state["collected_data"] = {}
                  session_state["step_history"] = []
                  del session_state["sub_step"]
                  return "Booking process restarted. How can I assist you?"
              else:
                  session_state["sub_step"] = "awaiting_change"
                  return "What would you like to change? You can say 'play', 'hall', 'showtime', 'tickets', or 'seats'."

    elif current_step == "payment":

        candidate_labels = [
            "credit_card",
            "cash",
            "unknown"
        ]
        method = llm_response(user_input, candidate_labels)
        print(method)

        if method in ["credit_card", "paypal", "bank_transfer"]:
            session_state["collected_data"]["payment_method"] = method
            session_state["current_step"] = "confirm_payment"
            return "payment process"
        elif method == "cash":
            booking_result = book_tickets(
                hall=session_state["collected_data"]["hall"],
                showtime=session_state["collected_data"]["time"],
                num_tickets=session_state["collected_data"]["num_tickets"],
                seats=session_state["collected_data"]["seats"],
                paid=False,
                name= session_state["collected_data"]["play"]
            )

            session_state["active"] = False
            session_state["current_step"] = None
            session_state["collected_data"] = {}
            session_state["step_history"] = []

            last_booking_id = sorted(theater_data['bookings'].keys(), key=int)[-1]
            last_booking = theater_data['bookings'][last_booking_id]
            print(last_booking)

            return f"Your booking is reserved. Please pay at the cashier to confirm.\n{booking_result}\nHow else can I help?"
        else:
            return "Invalid payment method. Please choose: Credit Card, PayPal, Bank Transfer, or pay in cash at the cashier."
    elif current_step == "confirm_payment":
        if user_input == "ok":
            session_state["collected_data"]["paid"] = True
            booking_result = book_tickets(
                    hall=session_state["collected_data"]["hall"],
                    showtime=session_state["collected_data"]["time"],
                    num_tickets=session_state["collected_data"]["num_tickets"],
                    seats=session_state["collected_data"]["seats"],
                    paid=True,
                    name= session_state["collected_data"]["play"]

                )
            session_state["current_step"] = "collect_email"
            session_state["booking_result"] = booking_result
            return f"Payment successful!\n{booking_result}\nPlease provide your email address to send the ticket:"
        else:
            session_state["active"] = False
            session_state["current_step"] = None
            session_state["collected_data"] = {}
            session_state["step_history"] = []
            return "Payment process canceled. How can I assist you?"

    elif current_step == "collect_email":
        FROM_EMAIL = "markoulikos@gmail.com"
        FROM_PASSWORD = "oced bmxw ylqv hxgz"

        email = user_input.strip()
        if is_valid_email(email):
            ticket_body = session_state.get("booking_result", "Here is your ticket info.")
            success = send_email(email, "Your Ticket Confirmation\n", ticket_body, FROM_EMAIL, FROM_PASSWORD)
            if success:
                session_state["active"] = False
                session_state["current_step"] = None
                session_state["collected_data"] = {}
                session_state["step_history"] = []
                session_state.pop("booking_result", None)
                return f"Ticket sent to {email}. Thank you for your purchase! How else can I help?"
            else:
                return "Sorry, I couldn't send the email. Please try again later."

        else:
            return "That doesn't look like a valid email address. Please enter a valid email to receive your ticket:"

    return "Error in booking process."

### Theater Info & Booking Utilities

- Show upcoming performances with summaries  
- Check booking status by ID  
- Cancel booking with email verification  
- Handle greetings and farewells  
- List available plays with showtimes  


In [8]:
def handle_performances_intent():

    performances = "\n".join(

        f"- {p['play']} in {p['hall']} at {', '.join(p['showtimes'])}"
        for p in theater_data["performances"]

    )

    summaries = {
        "Antigone": "Antigone tells the story of the daughter of Oedipus who defies King Creon’s order forbidding the burial of \n her brother Polynices. Driven by loyalty to family and divine law, Antigone chooses to bury her \n brother, knowing it will bring severe punishment. The play explores themes of civil disobedience, \n moral duty, and the conflict between individual conscience and state law.",
        "Electra": "Electra centers on the daughter of King Agamemnon who is consumed by grief and a desire for revenge after \n her father is murdered by her mother Clytemnestra and her lover Aegisthus. Electra longs for justice \n and works alongside her brother Orestes to avenge their father’s death.\n The play examines themes of revenge, justice, and family loyalty."
    }

    summary_text = "\n\n".join(
        f"**{title}**: {desc}" for title, desc in summaries.items()
    )

    return (
        f"Here are the upcoming performances:\n{performances}\n\n"

        f"🎭 **Play Summaries**:\n{summary_text}"
    )



def get_booking_status(booking_id):

    booking = theater_data["bookings"].get(int(booking_id), None)
    if booking:
        status = "Paid" if booking["paid"] else "Unpaid"
        return f"Booking ID: {booking_id}\nHall: {booking['hall']}\nShowtime: {booking['showtime']}\nTickets: {booking['num_tickets']}\nSeats: {', '.join(map(str, booking['seats']))}\nStatus: {status}"

    return "Booking not found."

def cancel_booking(booking_id, mail):
    print(booking_id,"  f. ", mail)
    booking = theater_data["bookings"].get(booking_id)

    if not booking:
        return f"Booking {booking_id} for {mail} not found."

    #Verify the email number matches
    if booking["email"] != mail:
        return "Email does not match the booking. Cancellation denied."

    key = (booking["hall"], booking["showtime"])
    theater_data["seat_availability"][key].update(booking["seats"])
    del theater_data["bookings"][booking_id]
    return f"Booking {booking_id} for {mail} canceled."




def handle_greeting():

    responses = [

        "Hello! How can I assist with your theater plans today?",
        "Hi! Ready to help with shows or bookings.",
        "Good day! What can I do for you?"
    ]

    return random.choice(responses)


def handle_goodbye():

    responses = [
        "Goodbye! Enjoy the show!",
        "Farewell! See you at the theater!",
        "Have a great day!"
    ]

    return random.choice(responses)

def list_available_plays(theater_data):
    performances = theater_data.get("performances", [])
    if not performances:
        return "No performances available at the moment."

    lines = ["Available performances:"]
    for p in performances:
        play = p["play"]
        hall = p["hall"]
        times = ", ".join(p["showtimes"])
        lines.append(f'- "{play}" at {hall} — Showtimes: {times}')

    lines.append("\nWhich play or hall would you like to choose?")
    return "\n".join(lines)

### Intent Recognition Function

This function  determines the user's intent from input text using rule-based keyword matching for common cases. If no rule matches, it falls back to a zero-shot classification via an external model to handle edge cases.


In [9]:
def recognize_intent(text):
    text_lower = text.lower()

    # Rule-based checks for known intents (most specific to general)
    if any(keyword in text_lower for keyword in ["cancel booking", "delete booking"]):
        return "cancel_booking"

    elif any(keyword in text_lower for keyword in ["check booking", "booking status"]):
        return "check_booking"

    elif any(keyword in text_lower for keyword in ["book", "reserve", "ticket"]):
        return "book_tickets"

    elif any(keyword in text_lower for keyword in ["performances", "shows", "plays", "what's on"]):
        return "get_performances"

    elif any(keyword in text_lower for keyword in ["human", "agent", "representative"]):
        return "human_support"

    elif any(greet in text_lower for greet in ["hi", "hello", "hey"]):
        return "greeting"

    elif any(farewell in text_lower for farewell in ["bye", "goodbye"]):
        return "end_of_chat"

    # Fallback to zero-shot classification for edge cases
    candidate_labels = [
        "book_tickets", "get_performances", "check_booking",
        "cancel_booking", "human_support", "greeting",
        "unknown", "end_of_chat"
    ]
    result = llm_response(text, candidate_labels)
    return result



# Chatbot Main Interaction Flow

This manages the dialogue with the user. It maintains state across turns to handle multi-step processes such as booking cancellation and status checking. The flow uses regex to extract booking IDs and emails and delegates tasks to specialized handlers based on recognized user intents. If the intent is unclear, it replies with varied fallback messages prompting clarification.


In [10]:
def chatbot(user_input):

     # Waiting for booking ID
    if session_state.get("waiting_for_booking_id") == "cancel":
        booking_id_match = re.search(r'\d+', user_input)
        if booking_id_match:
            session_state["waiting_for_booking_id"] = None
            session_state["waiting_for_phone_number"] = "cancel"
            session_state["booking_id_for_cancel"] = booking_id_match.group()
            return "Please provide the email associated with the booking to confirm cancellation."
        else:
            return "I didn't catch a valid booking ID. Please try again."

    # Waiting for phone number
    elif session_state.get("waiting_for_phone_number") == "cancel":
        email_match = re.search(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', user_input)
        if email_match:
            session_state["waiting_for_phone_number"] = None
            booking_id = session_state.get("booking_id_for_cancel")
            session_state["booking_id_for_cancel"] = None
            return cancel_booking(booking_id, email_match.group())
        else:
            return "I didn't catch a valid email. Please try again."

    # Booking check flow remains unchanged
    elif session_state.get("waiting_for_booking_id") == "check":
        booking_id_match = re.search(r'\d+', user_input)
        if booking_id_match:
            session_state["waiting_for_booking_id"] = None
            return get_booking_status(booking_id_match.group())
        else:
            return "I didn't catch a valid booking ID. Please try again."

    if session_state.get("active", False):
        return handle_booking_process(user_input)
    else:
        intent = recognize_intent(user_input)

        if intent == "greeting":
            return handle_greeting()

        elif intent == "book_tickets":
            session_state["active"] = True
            session_state["current_step"] = "plays"
            session_state["collected_data"] = {}
            session_state["step_history"] = []
            return handle_booking_process(user_input)

        elif intent == "get_performances":
            return handle_performances_intent()

        elif intent == "check_booking":
            booking_id_match = re.search(r'\d+', user_input)
            if booking_id_match:
                return get_booking_status(booking_id_match.group())
            else:
                session_state["waiting_for_booking_id"] = "check"
                return "Please provide your booking ID."


        elif intent == "cancel_booking":
            print(user_input)
            if session_state.get("waiting_for_phone_number") == "cancel":
                # User is now expected to provide the email number
                email_match = re.search(r'[\w.+-]+@[\w.-]+\.\w+', user_input)
                if email_match:
                    email = email_match.group()
                    booking_id = session_state.get("booking_id_for_cancel")
                    return cancel_booking(booking_id, email)
                else:
                    return "Please provide a valid email to confirm cancellation."

            booking_id_match = re.search(r'\d+', user_input)
            if booking_id_match:
                booking_id = booking_id_match.group()
                session_state["waiting_for_phone_number"] = "cancel"
                session_state["booking_id_for_cancel"] = booking_id
                return "Please provide the email associated with the booking to confirm cancellation."
            else:
                session_state["waiting_for_booking_id"] = "cancel"
                return "Please provide your booking ID to cancel."

        elif intent == "human_support":
            return "Connecting you to a human agent... Please call +30 (694) 5673 245"
        elif intent == "end_of_chat":
            session_state["active"] = False
            session_state["current_step"] = None
            session_state["collected_data"] = {}
            session_state["step_history"] = []
            return handle_goodbye()
        else:

            responses = [
                "Sorry, I didn't quite understand that. Can you please clarify your request? Here are some things I can help you with:\n"
                "- Book tickets\n"
                "- Check a booking status\n"
                "- Cancel a booking\n"
                "- Ask about upcoming performances\n"
                "- Get support from a human agent\n"
                "Please let me know what you'd like to do.",

                "I’m sorry, I didn’t catch that. Could you please rephrase your question or let me know what you need help with?",

                "I’m having trouble understanding that. Please make sure your request is clear. For example, "
                "you can ask me about booking tickets, checking bookings, canceling bookings, or finding performances.",

                "I didn’t quite understand that. Could you choose one of the following options?\n"
                "- Help me book tickets\n"
                "- Check my booking\n"
                "- Cancel my booking\n"
                "- Show me performances\n"
                "- Connect me to a human agent",

                "It looks like I couldn’t understand your request. Here’s how I can help:\n"
                "- If you're looking to book tickets, just let me know the show and time!\n"
                "- If you need to check or cancel a booking, please provide your booking ID.\n"
                "- Want to see performances? I can show you upcoming shows."
                "Let me know what you'd like to do!",

                "Oops! I didn’t quite understand your request. I'm here to help though! "
                "Can you tell me again what you're looking for? You can ask about tickets, performances, or anything else!"
            ]

            return random.choice(responses)



In [11]:
import smtplib
FROM_EMAIL = "markoulikos@gmail.com"
FROM_PASSWORD = "oced bmxw ylqv hxgz"
try:
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(FROM_EMAIL, FROM_PASSWORD)
    print("Login successful!")
    server.quit()
except Exception as e:
    print("Login failed:", e)


Login successful!


In [12]:
# To run it here:

# if __name__ == "__main__":

#     message = (
#         "Hi there! Welcome to the Pallas chatbot — your personal assistant for the Epidaurus Festival. 🌟\n\n"
#         "We have two gorgeous plays coming up on July 1st: Antigone and Electra.\n\n"
#         "How can I help you today? Whether you want information about the performances, book tickets, or cancel a reservation, I’m here to assist! 🎭"
#     )
#     print(message)
#     #print("Hi there! Welcome to the Pallas chatbot. I can help you get information about shows, book tickets, and even cancel them. How can I assist you today?")
#     session_state["active"] = False
#     while True:
#         user_input = input("\nYou: ")
#         if user_input.lower() in ['exit', 'quit']:

#             print("Goodbye!")
#             break

#         response = chatbot(user_input)
#         print(f"Bot: {response}")


In [13]:
!wget -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip
!./ngrok authtoken 2wrLqazFFpHGagzmQXtLQXGtG7X_2FxAuv7SCi7iHUjnDnhDp
!pip install pyngrok


--2025-05-31 23:06:52--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 99.83.220.108, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9462138 (9.0M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]   9.02M  4.01MB/s    in 2.2s    

2025-05-31 23:06:56 (4.01 MB/s) - ‘ngrok.zip’ saved [9462138/9462138]

Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Flask Chatbot API with Ngrok Tunneling

This script sets up a Flask server exposing a /chat POST endpoint for interacting with the chatbot. It maintains a simple global state to track repeated responses and switch to human support after repeated confusion. The server runs in a separate thread alongside an Ngrok tunnel to expose it publicly, enabling external access for testing or deployment.

- Sends a welcome message on receiving "START".
- Delegates other messages to the chatbot function.
- Switches to human support after 3 identical replies to avoid user frustration.


In [18]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import time

app = Flask(__name__)

sessions = {}

state = {
    "count": 0,
    "prev_reply": "",
    "human": False
}

# ========== Flask Endpoint ==========

@app.route('/chat', methods=['POST'])
def chat():
    global state

    if not state["human"]:
        data = request.get_json()
        if not data or 'message' not in data:
            return jsonify({"error": "No message provided"}), 400

        user_input = data['message']
        if user_input == 'START':
            session_state["active"] = False
            session_state["current_step"] = None
            session_state["collected_data"] = {}
            session_state["step_history"] = []
            reply = (
                "Hi there! Welcome to the Pallas chatbot — your personal assistant for the Epidaurus Festival. 🌟\n\n"
                "We have two gorgeous plays coming up on July 1st: Antigone and Electra.\n\n"
                "How can I help you today? Whether you want information about the performances, book tickets, or cancel a reservation, I’m here to assist! 🎭"
            )
        else:
            reply = chatbot(user_input)

        if state["prev_reply"] == reply:
            state["count"] += 1
        else:
            state["count"] = 0
            state["prev_reply"] = reply

        if state["count"] >= 2:
            state["human"] = True

    else:
        session_state["active"] = False
        session_state["current_step"] = None
        session_state["collected_data"] = {}
        session_state["step_history"] = []
        reply = "I'm sorry, I'm having trouble understanding your request. To help you better, please call our support at +30 (694) 5673 245 and speak with a representative."

        state["human"] = False

    return jsonify({"reply": reply})


def run_flask():
    app.run(host='0.0.0.0', port=5001)

def run_ngrok():
    ngrok.set_auth_token("2wrLqazFFpHGagzmQXtLQXGtG7X_2FxAuv7SCi7iHUjnDnhDp")

    tunnel = ngrok.connect(5001)
    print(f" * Ngrok URL: {tunnel.public_url}/chat")

if __name__ == "__main__":
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()

    time.sleep(2)  # Wait for initialization

    ngrok_thread = threading.Thread(target=run_ngrok, daemon=True)
    ngrok_thread.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nShutting down...")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit


 * Ngrok URL: https://2aaa-35-234-55-143.ngrok-free.app/chat


INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:37:19] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:37:22] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:37:36] "POST /chat HTTP/1.1" 200 -


{'sequence': 'antigone', 'labels': ['play: antigone', 'unknown', 'hall: hall 1', 'hall: hall 2', 'play: electra'], 'scores': [0.7327912449836731, 0.15965723991394043, 0.05333331227302551, 0.04161398857831955, 0.012604324147105217]}
play: antigone


...

{'sequence': 'antigone', 'labels': ['play: antigone', 'unknown', 'hall: hall 1', 'hall: hall 2', 'play: electra'], 'scores': [0.7327912449836731, 0.15965723991394043, 0.05333331227302551, 0.04161398857831955, 0.012604324147105217]}
play: antigone


INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:39:25] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:39:43] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:39:51] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:40:05] "POST /chat HTTP/1.1" 200 -


{'sequence': 'Lets say Antigone', 'labels': ['play: antigone', 'unknown', 'hall: hall 1', 'hall: hall 2', 'play: electra'], 'scores': [0.804338812828064, 0.11362848430871964, 0.043532293289899826, 0.030038658529520035, 0.008461758494377136]}
play: antigone


INFO:werkzeug:127.0.0.1 - - [31/May/2025 23:40:13] "POST /chat HTTP/1.1" 200 -



Shutting down...
